# Initialise

In [ ]:
# initiate all functions

import requests
import json
import pandas as pd
import numpy as np
import re


def get_response(username, type="chirp", limit=100):
    payload = json.dumps({"username": username, "limit": limit})
    headers = {
        "Content-Type": "application/json",
        "Cookie": "connect.sid=s%3AMykdjjS1VjZZOaQyIT7RhYG214TTMr8N.jIXuF6J9VFJrsGtqDp2Mfw41mBV14uGIyKbqXruSo68",
    }
    url = f"https://api.chirper.ai/v1/{type}"
    response = requests.request("GET", url, headers=headers, data=payload)
    if response.status_code == 200:
        try:
            return json.loads(response.text)
        except json.decoder.JSONDecodeError:
            return {}
    else:
        return {}


def get_activities(username, limit=100):
    json_response = get_response(username, type="activity", limit=limit)
    if len(json_response) == 0:
        return []
    else:
        return json_response["result"]["activities"]


def get_chirps(username, limit=100):
    json_response = get_response(username, type="chirp", limit=limit)
    if len(json_response) == 0:
        return []
    else:
        return json_response["result"]["chirps"]


def get_mentions(their_chirps_raw):
    their_mentions = []
    timestamps = []
    for chirp in their_chirps_raw:
        content = chirp["content"]
        mentions = re.findall(r"@\w+", content)
        mentions = [mention.rstrip(".").rstrip(",").lstrip("@") for mention in mentions]
        if len(mentions) > 0:
            timestamps += [chirp["createdAt"]] * len(mentions)
            their_mentions += mentions
    return their_mentions, timestamps


def get_engagement_df(chirper):
    its_activities = get_activities(chirper, limit=1000)

    its_mentions, its_mention_timestamps = get_mentions(get_chirps(chirper, limit=1000))
    its_likes = []
    its_dislikes = []
    its_reaction_timestamps = []
    for act in its_activities:
        if "to" in act.keys():
            if act["activity"] == "like":
                its_likes += [act["to"]["username"]]
                its_reaction_timestamps += [act["createdAt"]]
            elif act["activity"] == "dislike":
                its_dislikes += [act["to"]["username"]]
                its_reaction_timestamps += [act["createdAt"]]

    targets = its_mentions + its_likes + its_dislikes + ["NA"]
    engagement_type = (
        ["mention"] * len(its_mentions)
        + ["like"] * len(its_likes)
        + ["dislike"] * len(its_dislikes)
        + ["NA"]
    )
    timestamp = its_mention_timestamps + its_reaction_timestamps + ["NA"]

    its_engagemnt_df = pd.DataFrame(
        {
            "source": chirper,
            "target": targets,
            "engagement_type": engagement_type,
            "timestamp": timestamp,
        }
    )

    return its_engagemnt_df

In [ ]:
# get a list of all users to start our search from
raw_activities = get_activities("", 1000)
raw_users = [
    act["from"]["username"]
    for act in raw_activities
    if "from" in act.keys() and "username" in act["from"].keys()
]
raw_users = list(set(raw_users))
len(raw_users)

In [ ]:
# add the users we've already searched
full_engagement_network = pd.read_csv("data/full_engagement_network.csv")
raw_users += list(full_engagement_network["source"].unique())
raw_users += list(full_engagement_network["target"].unique())
raw_users = list(set(raw_users))
len(raw_users)

# Retrieve Engagement Network

## Get all raw engagements

In [ ]:
full_sample = raw_users.copy()
for idx, chirper in enumerate(full_sample):
    if idx % 100 == 0:
        print(idx, end=" ")
    if idx % 1000 == 0:
        print("")
        full_engagement_network = full_engagement_network.reset_index(drop=True)
        full_engagement_network.to_csv("data/full_engagement_network.csv")
    if chirper in set(full_engagement_network.source):
        continue
    else:
        its_engagemnt_df = get_engagement_df(chirper)
        full_engagement_network = pd.concat([full_engagement_network, its_engagemnt_df])

In [ ]:
# remove chinese characters in target if target has english characters
full_engagement_network["target"] = full_engagement_network["target"].apply(
    lambda x: np.NaN
    if pd.isnull(x)
    else x
    if re.search("[a-zA-Z]", str(x)) is None
    else re.sub(r"[\u4e00-\u9fff]+", "", str(x))
)
# drop na
full_engagement_network = full_engagement_network.dropna().reset_index(drop=True).copy()
full_engagement_network.to_csv("data/full_engagement_network.csv", index=False)